## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
import sys
sys.path.append("..")
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
import numpy as np
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 

city_data_df = pd.read_csv("../Weather_Database/WheatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hilo,US,19.73,-155.09,87.80,55,40,14.99,scattered clouds
1,1,Arraial Do Cabo,BR,-22.97,-42.03,70.12,73,70,9.08,broken clouds
2,2,Hovd,MN,48.01,91.64,32.04,79,0,2.71,clear sky
3,3,Salalah,OM,17.02,54.09,73.40,56,0,3.36,clear sky
4,4,Barrow,US,71.29,-156.79,17.60,72,1,26.40,clear sky


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
user_high = int(input("What is the maximum temperature you would enjoy?"))
user_low = int(input("What is the lowest temperature you would enjoy?"))

In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
user_df = city_data_df.loc[(city_data_df["Max Temp"] >= user_low) & (city_data_df["Max Temp"] <= user_high)]
user_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hilo,US,19.73,-155.09,87.80,55,40,14.99,scattered clouds
5,5,Mount Isa,AU,-20.73,139.50,87.80,16,0,11.41,clear sky
11,11,Sawakin,SD,19.11,37.33,80.46,70,0,4.52,clear sky
19,19,Ipora,BR,-16.44,-51.12,75.54,70,32,7.45,scattered clouds
28,28,Manicore,BR,-5.81,-61.30,77.72,77,44,1.59,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
659,659,Kloulklubed,PW,7.04,134.26,84.20,79,75,4.70,broken clouds
662,662,Qui Nhon,VN,13.77,109.23,77.18,91,100,8.57,light rain
663,663,Ciudad Bolivar,VE,8.12,-63.55,84.22,60,57,3.78,broken clouds
664,664,Nova Olinda Do Norte,BR,-3.89,-59.09,81.46,63,30,1.74,scattered clouds


In [6]:
# 4a. Determine if there are any empty rows.
print(user_df.isnull().sum())
print(user_df.count())
len(user_df)

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64
City_ID                193
City                   193
Country                193
Lat                    193
Lng                    193
Max Temp               193
Humidity               193
Cloudiness             193
Wind Speed             193
Current Description    193
dtype: int64


193

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = user_df.dropna()
len(clean_df)

193

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hilo,US,87.80,scattered clouds,19.73,-155.09,
5,Mount Isa,AU,87.80,clear sky,-20.73,139.50,
11,Sawakin,SD,80.46,clear sky,19.11,37.33,
19,Ipora,BR,75.54,scattered clouds,-16.44,-51.12,
28,Manicore,BR,77.72,scattered clouds,-5.81,-61.30,
29,Kalmunai,LK,75.72,broken clouds,7.42,81.82,
31,Avarua,CK,77.00,scattered clouds,-21.21,-159.78,
32,Atuona,PF,78.49,clear sky,-9.80,-139.03,
37,Almeirim,BR,84.65,scattered clouds,-1.52,-52.58,
38,Luba,GQ,77.00,few clouds,3.46,8.55,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        #hotel_df.loc[index, "Hotel Name"] = np.nan
        print("Hotel not found... skipping.")    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
print(hotel_df.isnull().sum())
print(len(hotel_df))
clean_hotel_df = hotel_df.dropna()
print(clean_hotel_df.isnull().sum())
print(len(clean_hotel_df))

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64
193
City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64
193


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [32]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """


<dl>
<dt><b>Hotel Name</b></dt>{Hotel Name}
<dt><b>City</b></dt>{City}
<dt><b>Country</b></dt>{Country}
<dt><b>Current Weather</b></dt>{Current Description}
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]
# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [33]:
# 11a. Add a marker layer for each city to the map. 
locations = locations
fig = gmaps.figure(center=(10.0, 30.0), zoom_level=1.8)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))